In [1]:
from price import *
import binomial as bi
from zero import *
import numpy as np
import pandas as pd

In [2]:
def beautify(__model, __round=4):
    string = "| "
    layer = 0
    indent = 2
    size = __model.size
    value_list = []
    
    cnt = 1
    for li in __model.as_list():
        append_list = [li[0]]
        i=1
        while i<len(li):
            if li[i]!=li[i-1]:
                append_list.append(li[i])
            i+=1
        while len(append_list) < cnt:
            append_list.append(append_list[0])
        cnt+=1
        value_list.append(append_list)
    
    while layer < size:
        step = layer
        while step < size:
            value = str(round(value_list[step][layer], 4))
            string+=value
            string+=" | "
            step += 1
        indent += len(str(round(value_list[layer][layer], 4)))
        if layer < size-1:
            string += "\n"+(" "*indent)+" | "
        layer += 1
    return string

In [3]:
# Daily Treasury Yield Curve Rate in 02/15/19
r1 = 0.0012
r2 = 0.0041
r3 = 0.0070

mp1 = round(100*np.exp(-r1*1.0), 4)
mp2 = round(100*np.exp(-r2*1.0), 4)
mp3 = round(100*np.exp(-r3*1.0), 4)
data = [
    [100, 1.00, 0, mp1],
    [100, 2.00, 0, mp2],
    [100, 3.00, 0, mp3]
]
print(data)
fr = list(map(lambda x: round(x, 4), forward_rate(data, 1.0)))
print(fr)

[[100, 1.0, 0, 99.8801], [100, 2.0, 0, 99.5908], [100, 3.0, 0, 99.3024]]
[0.0012, 0.0029, 0.0029]


In [4]:
market_data = pd.DataFrame({
    "Maturity": [1.0, 2.0, 3.0],
    "Price": [mp1, mp2, mp3],
    "Yield": [r1, r2, r3],
    "ForwardRate": fr
})

print(market_data)

   Maturity    Price   Yield  ForwardRate
0       1.0  99.8801  0.0012       0.0012
1       2.0  99.5908  0.0041       0.0029
2       3.0  99.3024  0.0070       0.0029


In [5]:
# calculate volatility
# Link: https://financetrain.com/how-to-calculate-interest-rate-volatility
# Yield data from 10/01/21 to 10/15/21
import math
yield_data = pd.DataFrame(
    {
        "1yr": [0.09, 0.09, 0.09, 0.10, 0.10, 0.09, 0.10, 0.11, 0.10, 0.12],
        "2yr": [0.27, 0.27, 0.28, 0.30, 0.32, 0.32, 0.35, 0.37, 0.36, 0.41],
        "3yr": [0.49, 0.52, 0.54, 0.55, 0.57, 0.59, 0.64, 0.66, 0.62, 0.70]
    }
)

def get_vol(m):
    Xt = []
    sum_x = 0
    i = 0
    n = len(yield_data[m])
    while i < len(yield_data[m]):
        value = yield_data[m][i]
        sum_x += value
        Xt.append(value)
        i += 1
    avg = sum_x/n
    var = 0
    for x in Xt:
        var += ((x-avg)**2)/(n-1)
    return var*math.sqrt(250)

vol2 = get_vol("2yr")
print("volatility in 2yr maturity: "+str(vol2))
vol3 = get_vol("3yr")
print("volatility in 3yr maturity: "+str(vol3))

volatility in 2yr maturity: 0.034872895307967944
volatility in 3yr maturity: 0.06985119987127486


In [6]:
from scipy.optimize import fsolve

vol = vol2
step1 = lambda t: [r1+t*1.0+vol*1.0**0.5, r1+t*1.0-vol*1.0**0.5]
get_irm = lambda t: bi.model([[r1], step1(t)])
get_bpm = lambda t: bond_model(100, get_irm(t), [0.5], 1.0)
price_error = lambda t: (get_bpm(t).value(0, 0)-mp2)**2

result = fsolve(price_error, 0.00)
print("Get result: " + str(result))
theta1 = result[0]
print("theta1 is " + str(theta1))
print("1step ir "+str(step1(theta1)))
irm = get_irm(theta1)
bpm = get_bpm(theta1)
print(beautify(irm, 4))
print()
print(beautify(bpm, 4))

Get result: [0.00230833]
theta1 is 0.002308331349889756
1step ir [0.0383812266578577, -0.031364563958078184]
| 0.0012 | 0.0384 | 
         | -0.0314 | 

| 99.5908 | 96.2346 | 100 | 
          | 103.1862 | 100 | 
                  | 100 | 


In [7]:
vol = vol3
r10 = irm.value(1, 0)
r11 = irm.value(1, 1)
step2 = lambda t: [r10+t*1.0+vol*1.0**0.5, r10+t*1.0-vol*1.0**0.5, r11+t*1.0-vol*1.0**0.5]
get_irm = lambda t: bi.model([[r1], step1(theta1), step2(t)], simplify=True)
get_bpm = lambda t: bond_model(100, get_irm(t), [0.5, 0.5], 1.0)
price_error = lambda t: (get_bpm(t).value(0, 0)-mp3)**2

result = fsolve(price_error, 0.00)
theta2 = result[0]
print("theta2 is " + str(theta2))
print("2step ir "+str(step2(theta2)))
irm = get_irm(theta2)
bpm = get_bpm(theta2)
print(beautify(irm, 4))
print()
print(beautify(bpm, 4))

theta2 is 0.0251343929445535
2step ir [0.13336681947368606, -0.006335580268863661, -0.07608137088479955]
| 0.0012 | 0.0734 | 0.1334 | 
         | -0.0663 | -0.0063 | 
                | -0.0761 | 

| 99.3024 | 87.4207 | 87.5144 | 100 | 
          | 111.4225 | 100.6356 | 100 | 
                  | 107.905 | 100 | 
                         | 100 | 


In [8]:
# get log volatility
def get_log_vol(m):
    Xt = []
    sum_x = 0
    i = 1
    n = len(yield_data[m])-1
    while i < len(yield_data[m]):
        value = np.log(yield_data[m][i]/yield_data[m][i-1])
        sum_x += value
        Xt.append(value)
        i += 1
    avg = sum_x/n
    var = 0
    for x in Xt:
        var += ((x-avg)**2)/(n-1)
    return var*math.sqrt(250)
vol2 = get_log_vol("2yr")
print("volatility in 2yr maturity: "+str(vol2))
vol3 = get_log_vol("3yr")
print("volatility in 3yr maturity: "+str(vol3))

volatility in 2yr maturity: 0.03881996483241765
volatility in 3yr maturity: 0.03917919844393071


In [9]:
from scipy.optimize import minimize
vol = vol2
def get_pe1(t):
    step1 = [np.log(r1)+t*1.0+vol*1.0**0.5, np.log(r1)+t*1.0-vol*1.0**0.5]
    step1 = [np.exp(i) for i in step1]
    irm = bi.model([[r1], step1])
    bpm = bond_model(100, irm, [0.5], 1.0)
    return (bpm.value(0, 0)-mp2)**2

result = minimize(get_pe1, 0.00)
print("Get result: " + str(result))
theta1 = result.x[0]
print("theta1 is " + str(theta1))
s1 = [np.log(r1)+theta1*1.0+vol*1.0**0.5, np.log(r1)+theta1*1.0-vol*1.0**0.5]
s1 = [np.exp(i) for i in s1]
irm = bi.model([[r1], s1])
bpm = bond_model(100, irm, [0.5], 1.0)
print(beautify(irm, 4))
print(beautify(bpm, 4))

Get result:       fun: 1.5227253358298786e-10
 hess_inv: array([[6.27113687]])
      jac: array([7.13033848e-06])
  message: 'Optimization terminated successfully.'
     nfev: 14
      nit: 3
     njev: 7
   status: 0
  success: True
        x: array([0.88181702])
theta1 is 0.8818170237808484
| 0.0012 | 0.003 | 
         | 0.0028 | 
| 99.5908 | 99.6991 | 100 | 
          | 99.7216 | 100 | 
                 | 100 | 


In [10]:
r10 = irm.value(1, 0)
r11 = irm.value(1, 1)
vol = vol3
step2 = lambda t: [np.exp(np.log(r10)+t*1.0+vol*1.0**0.5), np.exp(np.log(r10)+t*1.0-vol*1.0**0.5), np.exp(np.log(r11)+t*1.0-vol*1.0**0.5)]
get_irm = lambda t: bi.model([[r1], s1, step2(t)], simplify=True)
get_bpm = lambda t: bond_model(100, get_irm(t), [0.5, 0.5], 1.0)
price_error = lambda t: (get_bpm(t).value(0, 0)-mp3)**2

result = minimize(price_error, 0.00)
theta2 = result.x[0]
print("theta2 is "+str(theta2))
print("2step ir "+str(step2(theta2)))
irm = get_irm(theta2)
bpm = get_bpm(theta2)
print(beautify(irm, 4))
print(beautify(bpm, 4))

theta2 is -0.0007851223428195755
2step ir [0.0031310005609627286, 0.0028950263839792314, 0.0026787607877497436]
| 0.0012 | 0.003 | 0.0031 | 
         | 0.0028 | 0.0029 | 
               | 0.0027 | 
| 99.3024 | 99.3992 | 99.6874 | 100 | 
          | 99.4441 | 99.7109 | 100 | 
                 | 99.7325 | 100 | 
                        | 100 | 


$$ Forward Rate = [(1 + S_1)^n_1/(1 + S_2)^n_2]^1/(n_1-n_2)-1 $$

$$Daily Volatility = \sum[(s_2/s_1-1)-avg]^2$$

$$AnnualVolatility = DailyVolatility\sqrt{252}$$

$$ r_1,_0 = 0.12\% + \theta_0 \times \delta + \sigma(\delta)^2$$
$$ r_1,_1 = 0.12\% + \theta_0 \times \delta - \sigma(\delta)^2$$


$$ 99.5908 = e^{-r_0\times\delta} \times(0.5 \times e^{-r_{1,1}}\times\delta)\times100$$

$$AnnualVolatility = \sum[ln(s_2/s_1)-avg]^2\sqrt{252}$$

$$ r_1,_0 = exp(ln(0.12)\% + \theta_0 \times \delta + \sigma(\delta)^2)$$
$$ r_1,_1 = exp(ln(0.12)\% + \theta_0 \times \delta - \sigma(\delta)^2)$$